In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import joblib

# Загрузим обработанный датасет
data_path = 'data/processed/'  # Путь к папке с .txt-книгами
book_filenames = ['book1.txt', 'book2.txt', 'book3.txt']  # Пример списка файлов
corpus = []
for filename in book_filenames:
    with open(data_path + filename, 'r', encoding='utf-8') as file:
        text = file.read()
        corpus.append(text)

# Создадим токенизатор BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Преобразуем текст в токены
input_ids = []
for line in corpus:
    encoded_text = tokenizer.encode(line, add_special_tokens=True)
    input_ids.append(encoded_text)

# Подготовим данные для обучения
max_sequence_length = max([len(seq) for seq in input_ids])
X = torch.tensor([seq + [0] * (max_sequence_length - len(seq)) for seq in input_ids])
y = torch.tensor([0, 1, 0])  # Пример целевой переменной (0 - не интересно, 1 - интересно)

# Создадим модель BERT для классификации
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Обучим модель
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
for epoch in range(5):
    for batch in dataloader:
        optimizer.zero_grad()
        input_batch, labels = batch
        outputs = model(input_batch, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Сохраним модель
joblib.dump(model, 'models/bert_model/bert_model.pth')

print("Модель BERT обучена и сохранена в 'models/bert_model'.")
